In [1]:
# Load the dataset

import pandas as pd

df = pd.read_csv("../data/4/train.csv")

In [2]:
# Examine the first 5 rows to get a general idea about the data
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Describe the numerical features (mean, min, max etc.)
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
# See what the types of the features are as well as the number of non-null items per column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
# Generate a report about the dataset
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)

In [6]:
profile.to_file("report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 66.66it/s]


In [7]:
# Separate the inputs from the output
X = df.loc[:, df.columns != "Survived"]
y = df["Survived"]

In [8]:
# Drop the PassengerId and Name columns because they are unique and therefore worthless for our analysis 
X = X.drop("PassengerId", axis=1)
X = X.drop("Name", axis=1)

# Substitute the missing values with some predetermined values
X.loc[X["Age"].isna(), "Age"] = X["Age"].mean()
X.loc[X["Cabin"].isna(), "Cabin"] = "Missing"

In [9]:
# We check what else we can drop
X.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,22.0,1,0,A/5 21171,7.2500,Missing,S
1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,Missing,S
3,1,female,35.0,1,0,113803,53.1000,C123,S
4,3,male,35.0,0,0,373450,8.0500,Missing,S


In [10]:
# We also drop the Ticket column
X = X.drop("Ticket", axis=1)
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,Missing,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,Missing,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,Missing,S


In [11]:
def get_dummies(df, column):
    # We drop one column from the generated columns to avoid the dummy variable trap
    dummies = pd.get_dummies(df[column], prefix=column, drop_first=True)
    return pd.concat([df, dummies], axis = 1).drop(column, axis=1)

In [12]:
# We generate dummies for the categorical variables (one-hot encoding)
X = get_dummies(X, "Sex")
X = get_dummies(X, "Cabin")
X = get_dummies(X, "Pclass")
X = get_dummies(X, "Embarked")

In [13]:
X.head()

,Age,SibSp,Parch,Fare,Sex_male,Cabin_A14,Cabin_A16,Cabin_A19,Cabin_A20,Cabin_A23,...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_Missing,Cabin_T,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,22.0,1,0,7.2500,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
1,38.0,1,0,71.2833,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,26.0,0,0,7.9250,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
3,35.0,1,0,53.1000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,35.0,0,0,8.0500,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1


In [14]:
# We can now fit an ML model to the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [15]:
X_train.shape

(712, 156)

In [16]:
X_test.shape

(179, 156)

In [17]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)

C:\Users\gery\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [18]:
y_hat = model.predict(X_test)

In [19]:
# We can measure the accuracy of the model on the test set
(y_test == y_hat).sum() / len(y_test)

0.8100558659217877

In [20]:
X_train

,Age,SibSp,Parch,Fare,Sex_male,Cabin_A14,Cabin_A16,Cabin_A19,Cabin_A20,Cabin_A23,...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_Missing,Cabin_T,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
331,45.500000,0,0,28.5000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
733,23.000000,0,0,13.0000,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
382,32.000000,0,0,7.9250,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
704,26.000000,1,0,7.8542,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
813,6.000000,4,2,31.2750,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,21.000000,0,0,7.6500,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
270,29.699118,0,0,31.0000,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
860,41.000000,2,0,14.1083,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
435,14.000000,1,2,120.0000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
# We can use scaling or normalization to scale/normalize the input values
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]])

<ipython-input-21-d333f71803a0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]])
C:\Users\gery\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [22]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [23]:
X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]])
y_hat = model.predict(X_test)

(y_test == y_hat).sum() / len(y_test)

<ipython-input-23-bc85a12d7536>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]])
C:\Users\gery\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


0.7821229050279329

In [24]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer()

X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]])

<ipython-input-24-01cc08194ded>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_train[X_train.columns[~X_train.columns.isin(["Sex_female","Sex_male"])]])
C:\Users\gery\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [25]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [26]:
X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]])
y_hat = model.predict(X_test)

(y_test == y_hat).sum() / len(y_test)

<ipython-input-26-bc85a12d7536>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]] = scaler.fit_transform(X_test[X_test.columns[~X_test.columns.isin(["Sex_female","Sex_male"])]])
C:\Users\gery\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


0.7877094972067039